Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2 : Set Covering using GA

## Task

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$, 
determine, if possible, $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$
such that each number between $0$ and $N-1$ appears in at least one list

$$\forall n \in [0, N-1] \ \exists i : n \in L_{s_i}$$

and that the total numbers of elements in all $L_{s_i}$ is minimum. 


Try to use a GA approach.

In [32]:
import logging
from collections import namedtuple
import random

In [33]:
POPULATION_SIZE = 1500
OFFSPRING_SIZE = 1000

NUM_GENERATIONS = 200

In [34]:
def distinct(list):
   result = []
   for l in list:
      if l not in result:
         result.append(l)
   return result

In [35]:
Individual = namedtuple("Individual", ["genome", "fitness"])

def coverage(genome, lists):                                        # How many numbers are covered
    values = set()
    for i in range(len(genome)): 
        if genome[i] == 1:
            values |= set(lists[i])
    return len(values)

def weight(genome, lists):                                          # Weight of the solution
    return sum(genome[i]*len(lists[i]) for i in range(len(lists)))

def compute_fitness(genome, lists):
    return (coverage(genome, lists), -weight(genome, lists))

def tournament(population, tournament_size=2): 
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness) 

def cross_over(g1, g2):
    cut = random.randint(0, len(g1))
    return g1[:cut] + g2[cut:]

def mutation(g):
    point = random.randint(0, len(g) - 1)
    return g[:point] + (1 - g[point],) + g[point + 1 :]

In [36]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

## Evolution

In [37]:
def my_genetic_algorithm(N, lists, population):
    for generation in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            m_rate = 0.3
            if random.random() < m_rate:                # MUTATION
                p = tournament(population)                  # promising genome
                o = mutation(p.genome)                      # promising genome mutated
            else:                                       # CROSSOVER
                p1 = tournament(population)                 # promising genome 1
                p2 = tournament(population)                 # promising genome 2
                o = cross_over(p1.genome, p2.genome)    

            f = compute_fitness(o, lists)
            offspring.append(Individual(o, f))


        population += offspring
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]

        best_so_far = population[0]
        if best_so_far.fitness[0] == -best_so_far.fitness[1] and best_so_far.fitness[0] == N:
            break

    print(f"#{N} : weight: {-best_so_far.fitness[1]}, bloat: {(-best_so_far.fitness[1]-N)/N*100}, #generations: {generation+1}")

In [38]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20, 100, 500, 1000]:
    population = list()
    lists = problem(N, 42)
    lists = distinct(lists)
   
    # try to create randomic population, this leads to worse results
    # for genome in [tuple([random.choice([1, 0]) for _ in range(len(lists))]) for _ in range(POPULATION_SIZE)]:
      #  population.append(Individual(genome, compute_fitness(genome, lists)))
    for genome in [tuple([0 for _ in range(len(lists))]) for _ in range(POPULATION_SIZE)]:
        population.append(Individual(genome, compute_fitness(genome, lists)))
    
    my_genetic_algorithm(N, lists, population)
    print("------------") 

#5 : weight: 5, bloat: 0.0, #generations: 3
------------
#10 : weight: 10, bloat: 0.0, #generations: 7
------------
#20 : weight: 24, bloat: 20.0, #generations: 200
------------
#100 : weight: 202, bloat: 102.0, #generations: 200
------------
#500 : weight: 1610, bloat: 222.00000000000003, #generations: 200
------------
#1000 : weight: 3600, bloat: 260.0, #generations: 200
------------
